# Stress analysis of a shear zone using MDM

Conclusion - the MDM model does not provide the sufficient flexibility to cover both the stress analysis in the elastic regime and the softening branch used to detect the localized cracks. It is not able to show the smooth response.

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, \
    DICAlignedGrid
from bmcs_shear.api import DICGridTri
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
# import collections
# collections.Iterable = collections.abc.Iterable

In [ ]:
test = 'B9_TV2'
dic_grid = DICGridTri(U_factor=100, dir_name=test, t=1, padding=40, d_x=3, d_y=3,
                     T_stepping='delta_T', n_T_max = 40)
dic_grid.read_beam_design()
dic_grid.trait_set(d_x=5, d_y=5);

The grid size with $d_x = 5$ mm and 
The $E$ modulus is set very low to attain the crack level in the range of

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, R=10, omega_threshold=0.2, verbose_eval=True)
dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                      epsilon_f=0.002, eps_max=0.01);

In [ ]:
dsf.tmodel_.interact()

In [ ]:
dsf.eval()

In [ ]:
delta_ab = np.eye(2)
sig_TEmab = np.array(dsf.sig_TEmab)
I1_TEm = np.einsum('aa,...aa->...', delta_ab, sig_TEmab)
s_TEmab = sig_TEmab - np.einsum('ab,...->...ab', delta_ab, I1_TEm) 
J2_TEm = 0.5*np.einsum('...ab,...ba->...', s_TEmab, s_TEmab)

In [ ]:
x_KL, y_KL = np.einsum('KLa->aKL', dsf.X_fe_KLa)

The resulting compression stress is very small because of a low value of $E$ modulus. 

In [ ]:
%matplotlib widget
from matplotlib import cm
fig = plt.figure(figsize=(11,3))
ax_rows = fig.subplots(3,3)
for (ax_comp, ax_o, ax_s), T in zip(ax_rows, [1, 10, -1]):
    I1_Em = np.copy(I1_TEm[T])
    I1_KL = dsf.transform_mesh_to_grid(I1_Em)
    I1_KL[I1_KL > 0] = 0
    vmin, vmax= -10, -0.1
    levels = np.linspace(vmin, vmax, 4)
    cs_comp = ax_comp.contourf( x_KL, y_KL, I1_KL, levels=levels, vmin=vmin, vmax=vmax, cmap='RdPu_r')
    ax_comp.axis('equal');
    ax_comp.axis('off');
    I1_KL = dsf.transform_mesh_to_grid(I1_Em)
    vmin, vmax = 1, 6
    levels = np.linspace(vmin, vmax, 6)
    ax_o.contourf( x_KL, y_KL, I1_KL, levels=levels, vmin=vmin, vmax=vmax, cmap='RdPu')
    ax_o.axis('equal');
    ax_o.axis('off');
    J2_Em = np.copy(J2_TEm[T])
    J2_KL = dsf.transform_mesh_to_grid(J2_Em)
    s = slice(0,None)
    cs_sig = ax_s.contourf( x_KL, y_KL, J2_KL, levels=levels, vmin=vmin, vmax=vmax, cmap='RdPu')
    ax_s.axis('equal');
    ax_s.axis('off');
fig.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.95, 0.1, 0.01, 0.8])
fig.colorbar(cs_sig, cax=cbar_ax)

fig.subplots_adjust(left=0.08)
cbar_ax = fig.add_axes([0.05, 0.1, 0.01, 0.8])
fig.colorbar(cs_comp, cax=cbar_ax)

# fig.colorbar(cs_sig) # cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap),
# cbar_sig = fig.colorbar(cm.ScalarMappable(norm=cs_sig.norm, cmap=cs_sig.cmap),
#                          ax=ax_s, #ticks=np.arange(0, max_fe_eps * 1.01, 0.005),
#                          orientation='horizontal')    